In [ ]:
!pip install gitpython
!pip install keras==2.3.1
!pip install plotly

In [ ]:
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam 
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from keras.layers import LSTM
from sklearn.model_selection import KFold, cross_val_score, train_test_split
import os
import json
import sklearn
import sys
import pickle
import re
import requests
import git
import pandas as pd
import numpy as np
import warnings
import itertools
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import statsmodels.api as sm
import matplotlib
matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
matplotlib.rcParams['text.color'] = 'k'
import plotly.offline as pyoff
import plotly.graph_objs as go
from sklearn.feature_extraction import text
from string import punctuation, printable
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
path_to_json = './ai-workflow-capstone/cs-train'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
json_files

In [ ]:
appended_data = []
for i in json_files:
    data = pd.read_json(path_to_json+'/'+i)
    data = data.rename(columns={'ID_STREAM': 'stream_id', 'VIEWED_TIMES': 'times_viewed', 'price' : 'total_price'})
    appended_data.append(data)
appended_data = pd.concat(appended_data)
appended_data.head()

In [ ]:
appended_data["date"] = appended_data["day"].astype(str) + '-' + appended_data["month"].astype(str) + '-' + appended_data["year"].astype(str)
appended_data['invoice'] = appended_data['invoice'].str.replace(r'\D', '').astype(int)
appended_data['date']= pd.to_datetime(appended_data['date'])
appended_data.head()

In [ ]:
appended_data.describe()

In [ ]:
print("unique values stream_id =",  appended_data['stream_id'].nunique())
print("unique values country =",  appended_data['country'].nunique())
print("unique values customer_id =",  appended_data['customer_id'].nunique())
print("uinique values invoice =",  appended_data['invoice'].nunique())

In [ ]:
Top10countries = appended_data.groupby(by=["country"]).sum()
Top10countries = Top10countries.sort_values(by=['total_price'], ascending=False)
Top10countries = Top10countries.drop(columns=['customer_id', 'invoice', 'year', 'month', 'day'])
Top10countries.head(12)

In [ ]:
n = 10
notintop10 = Top10countries.index.values.tolist()
del notintop10[:n]
print(notintop10)

In [ ]:
appended_data_top10 = appended_data
for c in notintop10:
    indexNames = appended_data_top10[(appended_data_top10['country'] == c)].index
    appended_data_top10.drop(indexNames , inplace=True)
appended_data_top10

In [ ]:
print("unique values stream_id =",  appended_data['stream_id'].nunique())
print("unique values country =",  appended_data['country'].nunique())
print("unique values customer_id =",  appended_data['customer_id'].nunique())
print("unique values invoice =",  appended_data['invoice'].nunique())

appended_data.country.unique()

In [ ]:
grouped_data = appended_data_top10.groupby(by=['country', 'date', 'year', 'month', 'day']).sum()
grouped_data = grouped_data.drop(columns=['invoice', 'customer_id'])
grouped_data.reset_index(inplace=True)
grouped_data.head(10)

In [ ]:
for country in grouped_data['country'].unique():
     grouped_data[grouped_data['country']==country].sort_values('date').plot.line(x='date',
                                                              y='total_price',
                                                              title=country)

In [ ]:
grouped = grouped_data.groupby(grouped_data.country) 
df_UK = grouped.get_group("United Kingdom") 
df_EIRE = grouped.get_group("EIRE")
df_Germany = grouped.get_group("Germany")
df_France = grouped.get_group("France")
df_Spain = grouped.get_group("Spain")
df_NL = grouped.get_group("Netherlands")
df_Portugal = grouped.get_group("Portugal")
df_Norway = grouped.get_group("Norway")
df_HK = grouped.get_group("Hong Kong")
df_Singh = grouped.get_group("Singapore")

In [ ]:
df_UK["yearmonth"] = df_UK["month"].astype(str) + '-' + df_UK["year"].astype(str)
df_UK['yearmonth']= pd.to_datetime(df_UK['yearmonth'])
df_UK = df_UK.groupby(by=["yearmonth"]).sum()

df_EIRE["yearmonth"] = df_EIRE["month"].astype(str) + '-' + df_EIRE["year"].astype(str)
df_EIRE['yearmonth']= pd.to_datetime(df_EIRE['yearmonth'])
df_EIRE = df_EIRE.groupby(by=["yearmonth"]).sum()

df_Germany["yearmonth"] = df_Germany["month"].astype(str) + '-' + df_Germany["year"].astype(str)
df_Germany['yearmonth']= pd.to_datetime(df_Germany['yearmonth'])
df_Germany = df_Germany.groupby(by=["yearmonth"]).sum()

df_France["yearmonth"] = df_France["month"].astype(str) + '-' + df_France["year"].astype(str)
df_France['yearmonth']= pd.to_datetime(df_France['yearmonth'])
df_France = df_France.groupby(by=["yearmonth"]).sum()

df_Spain["yearmonth"] = df_Spain["month"].astype(str) + '-' + df_Spain["year"].astype(str)
df_Spain['yearmonth']= pd.to_datetime(df_Spain['yearmonth'])
df_Spain = df_Spain.groupby(by=["yearmonth"]).sum()

df_NL["yearmonth"] = df_NL["month"].astype(str) + '-' + df_NL["year"].astype(str)
df_NL['yearmonth']= pd.to_datetime(df_NL['yearmonth'])
df_NL = df_NL.groupby(by=["yearmonth"]).sum()

df_Portugal["yearmonth"] = df_Portugal["month"].astype(str) + '-' + df_Portugal["year"].astype(str)
df_Portugal['yearmonth']= pd.to_datetime(df_Portugal['yearmonth'])
df_Portugal = df_Portugal.groupby(by=["yearmonth"]).sum()

df_Norway["yearmonth"] = df_Norway["month"].astype(str) + '-' + df_Norway["year"].astype(str)
df_Norway['yearmonth']= pd.to_datetime(df_Norway['yearmonth'])
df_Norway = df_Norway.groupby(by=["yearmonth"]).sum()

df_HK["yearmonth"] = df_HK["month"].astype(str) + '-' + df_HK["year"].astype(str)
df_HK['yearmonth']= pd.to_datetime(df_HK['yearmonth'])
df_HK = df_HK.groupby(by=["yearmonth"]).sum()

df_Singh["yearmonth"] = df_Singh["month"].astype(str) + '-' + df_Singh["year"].astype(str)
df_Singh['yearmonth']= pd.to_datetime(df_Singh['yearmonth'])
df_Singh = df_Singh.groupby(by=["yearmonth"]).sum()

In [ ]:
df_UK.reset_index(inplace=True)
df_UK

In [ ]:
df_UK = df_UK.set_index('yearmonth')
df_UK.index

In [ ]:
df_UK.isnull().sum()

In [ ]:
y = df_UK['total_price'].resample('MS').mean()
y['2017':]

In [ ]:
y.plot(figsize=(15, 6))
plt.show()

In [ ]:
grouped_datax = pd.get_dummies(grouped_data, columns=['country'], drop_first=True, prefix='country')
date_time = pd.to_datetime(grouped_datax.pop('date'), format='%d.%m.%Y %H:%M:%S')
plot_cols = ['total_price', 'times_viewed']
plot_features = grouped_datax[plot_cols]
plot_features.index = date_time
_ = plot_features.plot(subplots=True)

plot_features = grouped_datax[plot_cols][:480]
plot_features.index = date_time[:480]
_ = plot_features.plot(subplots=True)



In [ ]:
column_indices = {name: i for i, name in enumerate(grouped_datax.columns)}

n = len(grouped_datax)
train_df = grouped_datax[0:int(n*0.7)]
val_df = grouped_datax[int(n*0.7):int(n*0.9)]
test_df = grouped_datax[int(n*0.9):]

num_features = grouped_datax.shape[1]

In [ ]:
train_mean = train_df.mean()
train_std = train_df.std()

train_df = (train_df - train_mean) / train_std
val_df = (val_df - train_mean) / train_std
test_df = (test_df - train_mean) / train_std

In [ ]:
import seaborn as sns
df_std = (grouped_datax - train_mean) / train_std
df_std = df_std.melt(var_name='Column', value_name='Normalized')
plt.figure(figsize=(12, 6))
ax = sns.violinplot(x='Column', y='Normalized', data=df_std)
_ = ax.set_xticklabels(grouped_datax.keys(), rotation=90)

In [ ]:
grouped_datax.tail()

In [ ]:
dataset_target = grouped_datax['total_price']
dataset_data = grouped_datax

X_train, X_test, y_train, y_test = train_test_split(dataset_data, dataset_target, test_size=0.2,  random_state=2)

In [ ]:
df_UK = df_UK.drop(columns=['year', 'month', 'day'])
df_UK.reset_index(inplace=True)
df_diff_UK = df_UK.copy()
df_diff_UK['prev_sales'] = df_diff_UK['total_price'].shift(1)
#drop the null values and calculate the difference
df_diff_UK = df_diff_UK.dropna()
df_diff_UK['diff'] = (df_diff_UK['total_price'] - df_diff_UK['prev_sales'])
#create dataframe for transformation from time series to supervised
df_supervised_UK = df_diff_UK.drop(['prev_sales'],axis=1)
#adding lags
for inc in range(1,13):
    field_name = 'lag_' + str(inc)
    df_supervised_UK[field_name] = df_supervised_UK['diff'].shift(inc)
#drop null values
df_supervised_UK = df_supervised_UK.dropna().reset_index(drop=True)

# Import statsmodels.formula.api
import statsmodels.formula.api as smf
# Define the regression formula
model = smf.ols(formula='diff ~ lag_1 + lag_2 + lag_3 + lag_4 + lag_5 + lag_6', data=df_supervised)
# Fit the regression
model_fit = model.fit()
# Extract the adjusted r-squared
regression_adj_rsq = model_fit.rsquared_adj
df_supervised.drop(columns=['lag_7', 'lag_8', 'lag_9', 'lag_10', 'lag_11', 'lag_12'])
print(regression_adj_rsq)

In [ ]:
#import MinMaxScaler and create a new dataframe for LSTM model
from sklearn.preprocessing import MinMaxScaler
df_model = df_supervised.drop(['total_price','yearmonth'],axis=1)
#split train and test set
train_set, test_set = df_model[0:-6].values, df_model[-6:].values



In [ ]:
#apply Min Max Scaler
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler = scaler.fit(train_set)
# reshape training set
train_set = train_set.reshape(train_set.shape[0], train_set.shape[1])
train_set_scaled = scaler.transform(train_set)
# reshape test set
test_set = test_set.reshape(test_set.shape[0], test_set.shape[1])
test_set_scaled = scaler.transform(test_set)

In [ ]:
X_train, y_train = train_set_scaled[:, 1:], train_set_scaled[:, 0:1]
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test, y_test = test_set_scaled[:, 1:], test_set_scaled[:, 0:1]
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

In [ ]:
model = Sequential()
model.add(LSTM(4, batch_input_shape=(1, X_train.shape[1], X_train.shape[2]), stateful=True))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, nb_epoch=100, batch_size=1, verbose=1, shuffle=False)

In [ ]:
y_pred = model.predict(X_test,batch_size=1)
#for multistep prediction, you need to replace X_test values with the predictions coming from t-1

In [ ]:
#reshape y_pred
y_pred = y_pred.reshape(y_pred.shape[0], 1, y_pred.shape[1])
#rebuild test set for inverse transform
pred_test_set = []
for index in range(0,len(y_pred)):
    print(np.concatenate([y_pred[index],X_test[index]],axis=1))
    pred_test_set.append(np.concatenate([y_pred[index],X_test[index]],axis=1))
#reshape pred_test_set
pred_test_set = np.array(pred_test_set)
pred_test_set = pred_test_set.reshape(pred_test_set.shape[0], pred_test_set.shape[2])
#inverse transform
pred_test_set_inverted = scaler.inverse_transform(pred_test_set)

In [ ]:
#create dataframe that shows the predicted sales
result_list = []
sales_dates = list(df_UK[-7:].yearmonth)
act_sales = list(df_UK[-7:].total_price)
for index in range(0,len(pred_test_set_inverted)):
    result_dict = {}
    result_dict['pred_value'] = int(pred_test_set_inverted[index][0] + act_sales[index])
    result_dict['yearmonth'] = sales_dates[index+1]
    result_list.append(result_dict)
df_result = pd.DataFrame(result_list)
#for multistep prediction, replace act_sales with the predicted sales

In [ ]:
df_result

In [ ]:
#merge with actual sales dataframe
df_sales_pred = pd.merge(df_UK,df_result,on='yearmonth',how='left')
#plot actual and predicted
plot_data = [
    go.Scatter(
        x=df_sales_pred['yearmonth'],
        y=df_sales_pred['total_price'],
        name='actual'
    ),
        go.Scatter(
        x=df_sales_pred['yearmonth'],
        y=df_sales_pred['pred_value'],
        name='predicted'
    )
    
]
plot_layout = go.Layout(
        title='Sales Prediction'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)